In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")


In [6]:
from langchain_groq import ChatGroq
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [9]:
# Initialize Groq Chat Model
chat_model = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
# Set system instructions
system_message = SystemMessage(content="You are an AI assistant that answers questions in a helpful and detailed way.")

# Chat history
chat_history = [system_message]

def chat_with_ai(user_input):
    # Add user message
    chat_history.append(HumanMessage(content=user_input))

    # Get AI response
    response = chat_model.invoke(chat_history)

    # Add AI response to history
    chat_history.append(AIMessage(content=response.content))

    return response.content  # Return the response text

# Example chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break

    response = chat_with_ai(user_input)
    print(f"Chatbot: {response}")


Chatbot: Yes, I strive to provide helpful and detailed responses to questions to the best of my ability. I am designed to provide accurate and relevant information while maintaining a conversational and engaging tone. My goal is to make the process of obtaining information as seamless and efficient as possible. If you have any questions, feel free to ask!
Chatbot: Hello! How can I help you today? I'm here to answer any questions you might have and provide information on a wide range of topics. Let me know what's on your mind, and I'll do my best to assist you.
Chatbot: Yes, I'm glad to hear that! I'm here to make your life easier by providing quick and accurate answers to your questions. I can help with a variety of tasks, such as providing information on a topic, setting reminders, or even telling a joke to lighten the mood. Let me know how I can assist you today!
Chatbot: I'm an artificial intelligence, so I don't have feelings or emotions like humans do. However, I'm always ready an

In [14]:
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [31]:
# Load API Key
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize Chat Model
chat_model = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")

# Initialize chat history
chat_history = ChatMessageHistory()

# Initialize memory with the new API
memory = ConversationBufferMemory(chat_memory=chat_history, return_messages=True)

# Define the Prompt Template

prompt = PromptTemplate(
    input_variables=["chat_history","user_input"],
    partial_variables={"chat_history": lambda: memory.chat_memory.messages},
    template="""
        =========================
        📜 Conversation History
        =========================
        {chat_history}

        👤 User: {user_input}
        🤖 AI:
        """,
)

# Create the Chain with Memory
chatbot_chain = LLMChain(llm=chat_model,prompt=prompt,memory=memory)

def chat_with_ai(user_input):
    response = chatbot_chain.invoke({"user_input": user_input})  # No need to pass chat_history manually
    return response["text"]
# Example chat loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!👋")
        break

    response = chat_with_ai(user_input)
    print(f"Chatbot: {response}")

Chatbot: I'm sorry for the confusion, but as a text-based AI, I don't have the ability to remember previous interactions within our current conversation. I'm designed to respond to each input individually. However, I'm here to help! If you need assistance with something specific, feel free to ask. For instance, you might need help with a math problem, want to know the capital of a country, or have a question about a general topic. What can I assist you with today?
Chatbot: I'm glad there are no hard feelings! I'm here to help with any questions or problems you have, to the best of my ability. You can ask me a math problem, request information about a country's capital, inquire about a general topic, or anything else you might need assistance with. What can I help you with today?
Chatbot: Goodbye!👋


2️⃣ Add RAG (Answering from Your Own PDFs & Docs)

In [32]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [33]:
# Load and split documents
loader = PyPDFLoader("NIPS-2017-attention-is-all-you-need-Paper.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(docs)


In [40]:
# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

# Query the chatbot based on documents
def query_documents(query):
    results = vectorstore.similarity_search_with_score(query, k=3)
    return results

# Example usage

query = "What are some key contributions of Attention in NIPS 2017?"
results = query_documents(query)
# Unpacking properly
for document, score in results:
    print(f"Document Title: {document.metadata.get('title', 'Unknown')}")
    print(f"Score: {score}")
    print(f"Content: {document.page_content}\n")

Document Title: Attention is All you Need
Score: 1.086801528930664
Content: We are excited about the future of attention-based models and plan to apply them to other tasks. We
plan to extend the Transformer to problems involving input and output modalities other than text and
to investigate local, restricted attention mechanisms to efﬁciently handle large inputs and outputs
such as images, audio and video. Making generation less sequential is another research goals of ours.
The code we used to train and evaluate our models is available at https://github.com/

Document Title: Attention is All you Need
Score: 1.1613496541976929
Content: arXiv:1703.10722, 2017.
[19] Zhouhan Lin, Minwei Feng, Cicero Nogueira dos Santos, Mo Yu, Bing Xiang, Bowen
Zhou, and Yoshua Bengio. A structured self-attentive sentence embedding. arXiv preprint
arXiv:1703.03130, 2017.
[20] Samy Bengio Łukasz Kaiser. Can active memory replace attention? In Advances in Neural
Information Processing Systems, (NIPS), 2016.


In [47]:
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import os
from dotenv import load_dotenv

# Load API Key
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize Chat Model
chat_model = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")

# Memory for chat history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Load and split documents
loader = PyPDFLoader("NIPS-2017-attention-is-all-you-need-Paper.pdf")  # Change this to your document
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(docs)

# Create embeddings & vectorstore
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

# Create RAG Retriever
retriever = vectorstore.as_retriever()

# Define query classification logic
def is_document_related(user_input):
    """Decides if the user input is related to the document or not"""
    keywords = ["pdf", "document", "file", "paper", "report", "study", "section", "page"]
    return any(keyword in user_input.lower() for keyword in keywords)

# System message for general chat
system_message = SystemMessage(content="You are a helpful AI assistant. Respond clearly and concisely.")

# Chat history
chat_history = [system_message]

# Function to handle user input
def chat_with_ai(user_input):
    global chat_history

    # Check if the question is related to the document
    if is_document_related(user_input):
        response = ConversationalRetrievalChain.from_llm(
            llm=chat_model,
            retriever=retriever,
            memory=memory
        ).run({"question": user_input})
        response_text = response  # Ensure response_text is assigned
    else:
        # Normal chat response
        chat_history.append(HumanMessage(content=user_input))
        
        # Call chat model
        response_obj = chat_model(chat_history)  # This may return an object, not just a string
        
        # Extract response content
        if isinstance(response_obj, str):
            response_text = response_obj  # If it's a string, use it directly
        elif hasattr(response_obj, "content"):
            response_text = response_obj.content  # Extract content if it's an AIMessage
        elif isinstance(response_obj, dict) and "content" in response_obj:
            response_text = response_obj["content"]  # Extract if it's a dict
        else:
            response_text = str(response_obj)  # Fallback to string conversion
        
        # Append AI response to history
        ai_message = AIMessage(content=response_text)
        chat_history.append(ai_message)

    return response_text  # Now it's always defined


# Chat loop
print("Chatbot is ready! Type 'exit' to quit.")
while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        print("\nChatbot: Goodbye!")
        break

    response = chat_with_ai(user_input)
    print(f"\nChatbot: {response}")


Chatbot is ready! Type 'exit' to quit.

Chatbot: Hello! I'm here to help. How can I assist you today?

Chatbot: The provided context consists of several research papers related to natural language processing and machine translation, as well as a table summarizing variations of the Transformer architecture for English-to-German translation.

Based on the given information, it is not possible to summarize the entire PDF as I only have access to the provided context. However, I can give a brief summary of the key points from the provided context:

1. There are several research papers listed, some of which focus on attention mechanisms, neural machine translation, and summarization.
2. Self-attention, also known as intra-attention, is an attention mechanism that relates different positions of a single sequence to compute a representation of the sequence. It has been successful in tasks like reading comprehension, abstractive summarization, textual entailment, and learning task-independent 